In [2]:
import torch
import pandas as pd
from data_process import decode, encode

In [6]:
df = pd.read_csv("3Di-15.csv")
names = df["name"].tolist()
repr_3Di = df["3Di"].tolist()
df["len"] = df["3Di"].apply(lambda x: len(x))
df["len"].mean()

301.3333333333333

In [8]:
# first index into the protein randomly
# then within the protein get a block_size chunk for x and offset by 1 for the targets 
def get_block_xy(repr_3Di, block_size):
	protein_idx = torch.randint(len(repr_3Di), (1,)).item()
	protein_3Di = repr_3Di[protein_idx]
	block_idx = torch.randint(len(protein_3Di) - block_size, (1,)).item()
	X = protein_3Di[block_idx:block_idx+block_size]
	Y = protein_3Di[block_idx+1:block_idx+block_size+1]
	return X, Y

def get_train_val_split(repr_3Di, ratio=0.9):
	split = int(ratio*len(repr_3Di))
	train = repr_3Di[:split]
	val = repr_3Di[split:]
	return train, val

train, val = get_train_val_split(repr_3Di)

In [117]:
def get_batch(split, batch_size=8, block_size=64):
	xs = []
	ys = []
	for b in range(batch_size):
		x, y = get_block_xy(split, block_size)
		xs.append(encode(x))
		ys.append(encode(y))
	xs = torch.tensor(xs, dtype=torch.long)
	ys = torch.tensor(ys, dtype=torch.long)
	return xs, ys
x, y = get_batch(val)

tensor([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2, 12,  2,  2, 12,  2, 12, 12, 17, 17, 17, 17, 17,
        17,  9,  1,  1, 17,  9, 12, 17,  2,  2])